In [1]:
from ib_insync import *
import pandas as pd
import time
import nasdaqdatalink
import sys
from datetime import datetime, timedelta
import numpy as np

sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.sharadar_utils.sharadar_constants import FAMA_INDUSTRY
nasdaqdatalink.ApiConfig.api_key = 'zxLNKQydu_qNXQ2tZ7vz'

# Industry Average

At the end we want a dataframe with each row is an industry and the columns are industry averages
                marketCap, ev, ps, pb, pe, roe, roa ...
Blank Checks 
Real Estate
Life Insurance

Let's Create our Empty Dataframe

In [2]:
# columns = ["ev", "marketcap", "evebit", "ebitda",
#            "pb", "pe", "ps"]
# index = FAMA_INDUSTRY

# result_df = pd.DataFrame(columns=columns, index=index)

# data = nasdaqdatalink.get_table('SHARADAR/SF1', dimension="MRT", ticker='AAPL,MSFT')
# df = pd.DataFrame(data)
# data

yesterday = datetime.now() - timedelta(days=3)
yesterday_date = yesterday.strftime('%Y-%m-%d')

daily = nasdaqdatalink.get_table('SHARADAR/DAILY', date=yesterday_date, qopts={"columns":['ticker', 'ev', 'evebit', 'evebitda', 'marketcap', 'pb', 'pe', 'ps']}, ticker='AAPL,MSFT')
daily




,ticker,ev,evebit,evebitda,marketcap,pb,pe,ps
None,,,,,,,,


In [3]:
universe = nasdaqdatalink.get_table('SHARADAR/TICKERS', table="SF1", qopts={"columns":['ticker' ,'name', 'isdelisted', 'exchange','category', 'famaindustry', 'industry', 'scalemarketcap', 'scalerevenue']})
df = pd.DataFrame(universe)
# We want filter anything thats delisted
universe_df = df[df['isdelisted'] == 'N']
universe_df


/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/nasdaqdatalink/get_table.py:38: UserWarning: To request more pages, please set paginate=True in your         nasdaqdatalink.get_table() call. For more information see our documentation:         https://github.com/Nasdaq/data-link-python/blob/main/FOR_ANALYSTS.md#things-to-note
  warnings.warn(Message.WARN_PAGE_LIMIT_EXCEEDED, UserWarning)


,ticker,name,isdelisted,exchange,category,famaindustry,industry,scalemarketcap,scalerevenue
None,,,,,,,,,
0,FMST,FOREMOST LITHIUM RESOURCE & TECHNOLOGY LTD,N,NASDAQ,Canadian Common Stock Primary Class,Non-Metallic and Industrial Metal Mining,Uranium,None,None
1,NNAGU,99 ACQUISITION GROUP INC,N,NASDAQ,Domestic Common Stock,Trading,Shell Companies,None,None
2,SEZL,SEZZLE INC,N,NASDAQ,Domestic Common Stock,Business Services,Software - Application,None,2 - Micro
3,SRM,SRM ENTERTAINMENT INC,N,NASDAQ,Domestic Common Stock,Recreation,Leisure,None,1 - Nano
4,SPGC,SACKS PARENTE GOLF INC,N,NASDAQ,Domestic Common Stock,Recreation,Leisure,None,1 - Nano
...,...,...,...,...,...,...,...,...,...
9981,TPHS,TRINITY PLACE HOLDINGS INC,N,NYSEMKT,Domestic Common Stock,Real Estate,Real Estate - Development,2 - Micro,2 - Micro
9988,SPI,SPI ENERGY CO LTD,N,NASDAQ,ADR Common Stock,Electronic Equipment,Solar,4 - Mid,2 - Micro
9993,CLLS,CELLECTIS SA,N,NASDAQ,ADR Common Stock,Pharmaceutical Products,Biotechnology,3 - Small,2 - Micro


## Loop through industries get list of tickers per industry

Here we want to loop through our industries and get a list of tickers for each industry

In [4]:
result_data = []

industries_dict = {}

for industry in FAMA_INDUSTRY:

    industry_df = universe_df[universe_df['famaindustry'] == industry]
    tickers_by_industry = industry_df["ticker"].astype(str).tolist()

    
    # ==========================================================
    # Daily Table (SHARADAR/DAILY)
    # ==========================================================
    yesterday = datetime.now() - timedelta(days=1)
    yesterday_date = yesterday.strftime('%Y-%m-%d')

    daily = nasdaqdatalink.get_table('SHARADAR/DAILY', date=yesterday_date, qopts={"columns":['ticker', 'ev', 'evebit', 'evebitda', 'marketcap', 'pb', 'pe', 'ps']}, ticker=tickers_by_industry)
    daily_df = pd.DataFrame(daily)

    mean_ev = daily_df['ev'].mean()
    mean_market_cap = daily_df['marketcap'].mean()

    positive_pb = daily_df[daily_df['pb'] > 0]
    mean_positive_pb = positive_pb['pb'].mean()

    positive_ps = daily_df[daily_df['ps'] > 0]
    mean_positive_ps = positive_ps['ps'].mean()

    positive_pe = daily_df[daily_df['pe'] > 0]
    mean_positive_pe = positive_pe['pe'].mean()

    positive_evebit = daily_df[daily_df['evebit'] > 0]
    mean_positive_evebit = positive_evebit['evebit'].mean()

    positive_evebita = daily_df[daily_df['evebitda'] > 0]
    mean_positive_evebita = positive_evebita['evebitda'].mean()

    row_values = {
        'industry': industry,
        'ev': mean_ev,
        'marketcap': mean_market_cap,
        'evebit': mean_positive_evebit,
        'evebitda': mean_positive_evebita,
        'pb': mean_positive_pb,
        'pe': mean_positive_pe,
        'ps': mean_positive_ps
    }

    # ==========================================================
    # Metrics Table (SHARADAR/METRICS)
    # ==========================================================
    metrics = nasdaqdatalink.get_table('SHARADAR/METRICS',  qopts={"columns":['ticker', 'beta1y', 'beta5y', 'dividendyieldforward', 'dividendyieldtrailing', 'return1y', 'return5y', 'volume', 'volumeavg1m', 'volumeavg3m']}, ticker=tickers_by_industry)
    metrics_df = pd.DataFrame(metrics)

    row_values['beta1y'] = metrics_df['beta1y'].mean()
    row_values['beta5y'] = metrics_df['beta5y'].mean()
    row_values['dividendyieldforward'] = metrics_df['dividendyieldforward'].mean()
    row_values['dividendyieldtrailing'] = metrics_df['dividendyieldtrailing'].mean()
    row_values['return1y'] = metrics_df['return1y'].mean()
    row_values['return5y'] = metrics_df['return5y'].mean()
    row_values['volume'] = metrics_df['volume'].mean()
    row_values['volumeavg1m'] = metrics_df['volumeavg1m'].mean()
    row_values['volumeavg3m'] = metrics_df['volumeavg3m'].mean()

    # ==========================================================
    # Fundamentals Table (SHARADAR/SF!)
    # ==========================================================

    # We will derive growth rates from trailing twelve months

    # We need grab growth rates which will be derived from the history of metrics such as revenue
    # We also need to grab the latest metrics which just takes the most recent observaton
    # Growth Rates involves ttm of full history (Revenue, EPS, Net Income, Operating Income, Margin Frowth Rates, Dividend Growth Rate, Free Cash Flow Growth Rate, Asset Growth Rate, Book Value Growth Rate, ROE growth rate, roa growth rate)
    # Most recent reported value

    full_figure_list = ['revenue', 'netinc', 'bvps', 'rnd', 'fcf', 'equity', 'assetturnover', 'currentratio', 'de', 'divyield', 'eps', 'fcfps', 'grossmargin', 'ebitdamargin', 'sps', 'ros', 'roic', 'roe', 'roa', 'payoutratio']
    latest_observation_list = ['bvps', 'assetturnover', 'currentratio', 'de', 'divyield', 'eps', 'fcfps', 'grossmargin', 'ebitdamargin', 'sps', 'ros', 'roic', 'roe', 'roa', 'payoutratio']
    
    
    aggregated_metrics = {metric: [] for metric in latest_observation_list}
    # We want to include average growth rates for the full figure list
    growth_rates = {metric: [] for metric in full_figure_list}

    print(growth_rates)

    # Loop through current tickers
    for ticker in tickers_by_industry:
        fundamental_data = nasdaqdatalink.get_table('SHARADAR/SF1', dimension="MRT", qopts={"columns":full_figure_list}, ticker=ticker)    
        df = pd.DataFrame(fundamental_data)

        if not df.empty:  
            fund_df = df.iloc[::-1]

            # Grab values that we want the most recent observation and add it to the aggregated_metrics dictionary
            first_row = fund_df.iloc[-1]
            for metric in latest_observation_list:
                value = first_row[metric]
                if (value and value > 0):
                    aggregated_metrics[metric].append(value)

            # Calculate simple growth rates for each metric and append to growth_rates dictionary

            for metric in full_figure_list:
                column = fund_df[metric]
                filtered_columns = column.dropna()

                # Filter out zero or near-zero values
                filtered_columns = filtered_columns[filtered_columns > 0.001]  # Adjust the threshold as needed


                # We want to calculate only companies that have a significant history so our
                # average growth rates are based off at least 5 years. We are using TTM so we
                # are calculated the TTM each quarter 4 * 5 = 20 entries
                if (len(filtered_columns) > 19):
                    simple_rates = (filtered_columns - filtered_columns.shift(1)) / filtered_columns.shift(1)
                    average_growth_rate = np.nanmean(simple_rates)
                    if not np.isnan(average_growth_rate):
                        growth_rates[metric].append(average_growth_rate)


    for metric, values in aggregated_metrics.items():
        row_values[metric] = np.mean(values)

    for metric, rates in growth_rates.items():
        row_values[f'{metric}_average_growth_rate'] = np.mean(rates)

    # Append row_values to result_data
    result_data.append(row_values)


result_df = pd.DataFrame(result_data)



# Write DataFrame to CSV file
csv_file_path = 'industry_metrics.csv'
result_df.to_csv(csv_file_path, index=False)
industries_dict



{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps':

/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps':

/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}


...

/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps':

/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}
{'revenue': [], 'netinc': [], 'bvps': [], 'rnd': [], 'fcf': [], 'equity': [], 'assetturnover': [], 'currentratio': [], 'de': [], 'divyield': [], 'eps': [], 'fcfps': [], 'grossmargin': [], 'ebitdamargin': [], 'sps': [], 'ros': [], 'roic': [], 'roe': [], 'roa': [], 'payoutratio': []}


{}